# Generate Some Offsets

<br>

**Step 1:** Get Sentinel-1 data <br>
**Step 2:** Get aux files <br>
**Step 3:** Get orbits files <br>
**Step 4:** Write XML files <br>
**Step 5:** Run `topsApp.py`

## Step 1: Getting Sentinel-1 Data

Downloads Sentinl-1 SLC bursts from Alaska Satellite Facility using the package asf_search.

In [1]:
import os
import asf_search as asf
from datetime import datetime
import pandas as pd
# import calplot
from zipfile import ZipFile
import getpass

from shapely.geometry import Polygon
import geopandas as gpd
from glob import glob
import subprocess

os.chdir('offsets/')  # Move to offsets directory

/home/jovyan/envs/crevasse-advection/lib/python3.13/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


### Get all SLC scenes that cover Shirase Glacier

Scenes that cover all of shirase: 
- 830
- 834
- 936
- 938
- 939

In [4]:
aoi = 'POLYGON((38.0336 -69.7358,38.0336 -70.4952,39.6985 -70.4952,39.6985 -69.7358,38.0336 -69.7358))'
n_files = int(input('number of files: '))

if n_files:
    opts = {
        'platform':'S1',
        'start':str(input('Start Date: ')),
        'processingLevel':'SLC',
        'frame':[830, 834, 936, 938, 939],
    }
    results = asf.search(intersectsWith=aoi, **opts)[-n_files:]

number of files:  0


### Start ASF Session and Download the .ZIP files

In [5]:
session = asf.ASFSession()

username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

results.download(path='SAFE/', session=user_pass_session)

Username: jackplogan
Password: ········


Success!


NameError: name 'results' is not defined

### Unzip the SAFE files, remove ZIP files

In [3]:
# Get all zip files from SAFE dir
zips = glob('SAFE/*.zip')

# Unzip the files
for zip_file in zips:
    with ZipFile(zip_file, 'r') as zf:
        zf.extractall('SAFE/')

    os.remove(zip_file)

## Step 4: Orbit and AUX Files

For dense offsets, we need the precise orbits and auxillary files for each individual SAFE.

In [6]:
%%bash

wget -nc https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
chmod +x fetchOrbit.py

#https://github.com/isce-framework/isce2/blob/main/contrib/stack/topsStack/fetchOrbit.py
# change lines 134 and 135 with your username and password

File ‘fetchOrbit.py’ already there; not retrieving.



In [7]:
# Get name of all safe files
safe_files = glob(f'SAFE/*.SAFE')

if len(safe_files) < 2:
    raise ValueError(f'You need at least 2 SAFE files, only found {len(safe_files)}')

In [8]:
# Download Orbit files to orbit folder
os.chdir(f'./orbits')

for file in safe_files:
    os.system(f'../fetchOrbit.py -i {file[5:-5]}')

Reference time:  2018-08-19 17:56:37
Satellite name:  S1A
using saved access token
Reference time:  2018-08-07 17:56:36
Satellite name:  S1A
using saved access token


### Orbit files acquired. Let's get the AUX files!

In [9]:
!echo "machine urs.earthdata.nasa.gov login {username} password {password}" > ~/.netrc
!chmod 0600 ~/.netrc

In [10]:
%%bash 
#!/bin/bash
URL=https://s1qc.asf.alaska.edu/aux_cal
cd ../aux
wget -r -l2 -nc -nd -np -nH -A SAFE $URL 

--2025-09-22 00:53:44--  https://s1qc.asf.alaska.edu/aux_cal
Resolving s1qc.asf.alaska.edu (s1qc.asf.alaska.edu)... 137.229.86.110
Connecting to s1qc.asf.alaska.edu (s1qc.asf.alaska.edu)|137.229.86.110|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://s1qc.asf.alaska.edu/aux_cal/ [following]
--2025-09-22 00:53:44--  https://s1qc.asf.alaska.edu/aux_cal/
Reusing existing connection to s1qc.asf.alaska.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘aux_cal.tmp’

     0K .....                                                  1.01G=0s

2025-09-22 00:53:44 (1.01 GB/s) - ‘aux_cal.tmp’ saved [5926]

Loading robots.txt; please ignore errors.
--2025-09-22 00:53:44--  https://s1qc.asf.alaska.edu/robots.txt
Reusing existing connection to s1qc.asf.alaska.edu:443.
HTTP request sent, awaiting response... 404 Not Found
2025-09-22 00:53:44 ERROR 404: Not Found.

Removing aux_cal.tmp since it should be r

## Step 4: Write XML Files

```xml
<?xml version="1.0" encoding="UTF-8"?>
<component name="reference">
    <property name="safe">../SAFE/S1A_IW_SLC__1SSH_20180807T175609_20180807T175636_023143_028391_AE45.SAFE</property>
    <property name="output directory">reference</property>
    <property name="orbit directory">../orbits</property>
    <property name="auxiliary data directory">../aux</property>
    <property name="polarization">hh</property>
</component>
```

```xml
<?xml version="1.0" encoding="UTF-8"?>
<component name="secondary">
    <property name="safe">../SAFE/S1A_IW_SLC__1SSH_20180819T175610_20180819T175637_023318_02893F_6998.SAFE</property>
    <property name="output directory">secondary</property>
    <property name="orbit directory">../orbits</property>
    <property name="auxiliary data directory">../aux</property>
    <property name="polarization">hh</property>
</component>
```

```xml
<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
    <component name="topsinsar">
        <property name="Sensor name">SENTINEL1</property>

        <!-- Scene XML files -->
        <component name="reference">
            <catalog>reference.xml</catalog>
        </component>
        <component name="secondary">
            <catalog>secondary.xml</catalog>
        </component>

        <!-- The swaths to process -->
        <property name="swaths">[2]</property>

        <!-- The region of interest -->
        <property name="region of interest">[-70.45441464, -69.88490745, 38.29553816,  39.83817435]</property>

        <!-- DEM for processing -->
        <property name="demFilename">../dem/dem.tiff</property>

        <!-- Unset all InSAR processing steps -->
        <property name="do interferogram">False</property>
        <property name="do ESD">False</property>
        <property name="do unwrap">False</property>
        <property name="do unwrap 2 stage">False</property>
        <property name="do ionosphere correction">False</property>
        <property name="geocode list">[]</property>

        <!-- Parameters for dense offsets -->
        <property name="do denseoffsets">True</property>
        <property name="Ampcor window width">256</property>
        <property name="Ampcor window height">64</property>
        <property name="Ampcor search window width">30</property>
        <property name="Ampcor search window height">10</property>
        <property name="Ampcor skip width">44</property>
        <property name="Ampcor skip height">8</property>

    </component>
</topsApp>
```


In [2]:
os.makedirs('../20180807-20180819/', exist_ok=True)
os.chdir('../20180807-20180819/')

In [3]:
topsApp = """<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
    <component name="topsinsar">
        <property name="Sensor name">SENTINEL1</property>

        <!-- Scene XML files -->
        <component name="reference">
            <catalog>reference.xml</catalog>
        </component>
        <component name="secondary">
            <catalog>secondary.xml</catalog>
        </component>

        <!-- The swaths to process -->
        <property name="swaths">[2]</property>

        <!-- The region of interest -->
        <property name="region of interest">[-70.45441464, -69.88490745, 38.29553816,  39.83817435]</property>

        <!-- DEM for processing -->
        <property name="demFilename">../dem/dem.tiff</property>

        <!-- Unset all InSAR processing steps -->
        <property name="do interferogram">False</property>
        <property name="do ESD">False</property>
        <property name="do unwrap">False</property>
        <property name="do unwrap 2 stage">False</property>
        <property name="do ionosphere correction">False</property>
        <property name="geocode list">[]</property>

        <!-- Parameters for dense offsets -->
        <property name="do denseoffsets">True</property>
        <property name="Ampcor window width">256</property>
        <property name="Ampcor window height">64</property>
        <property name="Ampcor search window width">30</property>
        <property name="Ampcor search window height">10</property>
        <property name="Ampcor skip width">44</property>
        <property name="Ampcor skip height">8</property>

    </component>
</topsApp>"""

reference = """<?xml version="1.0" encoding="UTF-8"?>
<component name="reference">
    <property name="safe">../SAFE/S1A_IW_SLC__1SSH_20180807T175609_20180807T175636_023143_028391_AE45.SAFE</property>
    <property name="output directory">reference</property>
    <property name="orbit directory">../orbits</property>
    <property name="auxiliary data directory">../aux</property>
    <property name="polarization">hh</property>
</component>"""

secondary = """<?xml version="1.0" encoding="UTF-8"?>
<component name="secondary">
    <property name="safe">../SAFE/S1A_IW_SLC__1SSH_20180819T175610_20180819T175637_023318_02893F_6998.SAFE</property>
    <property name="output directory">secondary</property>
    <property name="orbit directory">../orbits</property>
    <property name="auxiliary data directory">../aux</property>
    <property name="polarization">hh</property>
</component>"""

with open('topsApp.xml', 'w') as f:
    f.write(topsApp)

with open('reference.xml', 'w') as f:
    f.write(reference)

with open('secondary.xml', 'w') as f:
    f.write(secondary)

## Step 5: Run topsApp.py

Now we run from the command line

In [4]:
conda_path = '/home/jovyan'
isce_env = 'envs/isce2'

isce_home = f'{conda_path}/{isce_env}/lib/python3.8/site-packages/isce'
isce_stack = f'{conda_path}/{isce_env}/share/isce2'

isce_lib_path = "/home/jovyan/envs/isce2/lib"
os.environ["LD_LIBRARY_PATH"] = f"{isce_lib_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"
os.environ['ISCE_HOME'] = isce_home
os.environ['ISCE_STACK'] = isce_stack
os.environ['ISCE_ROOT'] = f'{conda_path}/{isce_env}/lib/python3.8/site-packages'

# Update PATH and PYTHONPATH
path_components = [
    f"{isce_home}/bin",
    f"{isce_home}/applications",
    f"{isce_stack}/topsStack",
    os.environ.get('PATH', '')
]
os.environ['PATH'] = ':'.join(filter(None, path_components))

pythonpath_components = [
    f'{conda_path}/{isce_env}/lib/python3.8/site-packages',
    isce_home,
    isce_stack,
    f"{isce_home}/applications",
    f"{isce_home}/components",
    os.environ.get('PYTHONPATH', '')
]
os.environ['PYTHONPATH'] = ':'.join(filter(None, pythonpath_components))

os.environ['OMP_NUM_THREADS'] = '8'

In [63]:
cmd = f"conda run -n isce2 topsApp.py topsApp.xml"

process = subprocess.Popen(
    cmd, 
    shell=True, 
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    env=os.environ,
    cwd=os.getcwd(),
)
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None:
        break
    if output:
        message = output.strip()
        print(message)


/home/jovyan/envs/isce2/lib/python3.8/site-packages/conda_package_streaming/package_streaming.py:25: UserWarning: zstandard could not be imported. Running without .conda support.
warnings.warn("zstandard could not be imported. Running without .conda support.")
/home/jovyan/envs/isce2/lib/python3.8/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
_warnings.warn("Install zstandard Python bindings for .conda support")
2025-09-22 01:07:12,720 - isce.insar - INFO - ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION =
ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION =
None
The currently supported sensors are:  ['SENTINEL1']
Could not extract swath 2 from ['../SAFE/S1A_IW_SLC__1SSH_20180807T175609_20180807T175636_023143_028391_AE45.SAFE']
Generated error:  No annotation xml file found in ../SAFE/S1A_IW_SLC__1SSH_20180807T175609_20180807T175636_02

## Step 6: Automate Offsets

Now that we've gone through this process, we can run the python script `run_isce2.py` from the command line. It will prompt you to input your NASA EarthData credentials, the number of SAFE files to download, and a start date. 

It will then download all the AUX and orbit files and run `topsApp.py`